# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)
# print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = "../output_data/cities_pd.csv"
cities_pd = pd.read_csv(cities_pd, encoding="ISO-8859-1")
cities_pd_clean = cities_pd.dropna()
cities_pd_clean.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,caravelas,-17.71,-39.25,83.05,61.0,32.0,12.75,BR,1.604076e+09
1,albany,42.60,-73.97,34.00,88.0,100.0,6.64,US,1.604076e+09
3,punta arenas,-53.15,-70.92,53.60,66.0,75.0,26.40,CL,1.604076e+09
4,chokurdakh,70.63,147.92,4.26,94.0,100.0,5.84,RU,1.604076e+09
5,tuatapere,-46.13,167.68,55.99,97.0,100.0,1.99,NZ,1.604076e+09


In [3]:
cities_pd_clean_temp = cities_pd_clean.loc[(cities_pd_clean["Max Temp"]<90)&(cities_pd_clean["Max Temp"]>80)]
cities_pd_clean_temp.count()

City          98
Lat           98
Lon           98
Max Temp      98
Humidity      98
Cloudiness    98
Windspeed     98
Country       98
Date          98
dtype: int64

In [4]:
### Humidity Heatmap
# * Configure gmaps.
# * Use the Lat and Lng as locations and Humidity as the weight.
# * Add Heatmap layer to map.

cities_pd_clean_temp_wind = cities_pd_clean_temp.loc[cities_pd_clean_temp["Windspeed"]<10]
cities_pd_clean_temp_wind.count()

City          58
Lat           58
Lon           58
Max Temp      58
Humidity      58
Cloudiness    58
Windspeed     58
Country       58
Date          58
dtype: int64

In [5]:
cities_pd_clean_temp_wind_clouds = cities_pd_clean_temp_wind.loc[cities_pd_clean_temp_wind["Cloudiness"]<5]
cities_pd_clean_temp_wind_clouds.head(8)

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
36,hithadhoo,-0.60,73.08,83.64,73.0,0.0,8.88,MV,1.604076e+09
60,abu dhabi,24.47,54.37,82.40,65.0,0.0,6.93,AE,1.604076e+09
349,dubai,25.26,55.30,84.20,54.0,0.0,4.70,AE,1.604076e+09
379,miramar,25.99,-80.23,88.00,62.0,1.0,3.36,US,1.604076e+09
474,rosamorada,22.12,-105.21,85.44,48.0,1.0,1.92,MX,1.604076e+09
546,acajutla,13.59,-89.83,86.00,55.0,0.0,3.36,SV,1.604076e+09
578,bara,10.37,10.73,83.91,31.0,2.0,1.10,NG,1.604076e+09
619,gusau,12.16,6.66,86.85,24.0,0.0,6.67,NG,1.604076e+09


In [6]:
perfect_vacation = cities_pd_clean_temp_wind_clouds 
perfect_vacation = perfect_vacation[["Country", "City", "Max Temp", "Humidity", "Cloudiness", "Windspeed", "Lat", "Lon", "Date"]]
perfect_vacation = perfect_vacation.rename(columns = {"Lon":"Lng"})
perfect_vacation = perfect_vacation.sort_values("City")
perfect_vacation.head(8)

### Create new DataFrame fitting weather criteria
# * Narrow down the cities to fit weather conditions.
# * Drop any rows will null values.

,Country,City,Max Temp,Humidity,Cloudiness,Windspeed,Lat,Lng,Date
60,AE,abu dhabi,82.40,65.0,0.0,6.93,24.47,54.37,1.604076e+09
546,SV,acajutla,86.00,55.0,0.0,3.36,13.59,-89.83,1.604076e+09
578,NG,bara,83.91,31.0,2.0,1.10,10.37,10.73,1.604076e+09
349,AE,dubai,84.20,54.0,0.0,4.70,25.26,55.30,1.604076e+09
619,NG,gusau,86.85,24.0,0.0,6.67,12.16,6.66,1.604076e+09
36,MV,hithadhoo,83.64,73.0,0.0,8.88,-0.60,73.08,1.604076e+09
379,US,miramar,88.00,62.0,1.0,3.36,25.99,-80.23,1.604076e+09
474,MX,rosamorada,85.44,48.0,1.0,1.92,22.12,-105.21,1.604076e+09


In [7]:
hotel_df = perfect_vacation
hotel_df = hotel_df.reset_index()
hotel_df = hotel_df.drop(columns=["index"])
hotel_df["Hotel Name"]= ""
hotel_df.head()

,Country,City,Max Temp,Humidity,Cloudiness,Windspeed,Lat,Lng,Date,Hotel Name
0,AE,abu dhabi,82.40,65.0,0.0,6.93,24.47,54.37,1.604076e+09,
1,SV,acajutla,86.00,55.0,0.0,3.36,13.59,-89.83,1.604076e+09,
2,NG,bara,83.91,31.0,2.0,1.10,10.37,10.73,1.604076e+09,
3,AE,dubai,84.20,54.0,0.0,4.70,25.26,55.30,1.604076e+09,
4,NG,gusau,86.85,24.0,0.0,6.67,12.16,6.66,1.604076e+09,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# geocoordinates
# target_coordinates = target_coordinates
target_search = "first hotel"
target_radius = 5000
target_type = "hotel"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    target_coordinate = f"{row ['Lat']} ,{row ['Lng']}"
    
#     print(target_coordinate)

# set up a parameters dictionary
    params = {
        "location": target_coordinate,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
#     print(response.url)
    hotel_name = response.json()
#     pprint(hotel_name['results'][0]['name'])
    
    try:
        hotel_value = hotel_name['results'][0]['name']
    except:    
        hotel_value = ""
#     print(hotel_value)\
    
    hotel_df.loc[index,"Hotel Name"] = hotel_value
               
hotel_df

,Country,City,Max Temp,Humidity,Cloudiness,Windspeed,Lat,Lng,Date,Hotel Name
0,AE,abu dhabi,82.40,65.0,0.0,6.93,24.47,54.37,1.604076e+09,Top Stars Hotel
1,SV,acajutla,86.00,55.0,0.0,3.36,13.59,-89.83,1.604076e+09,Villavela Hotel Boutique
2,NG,bara,83.91,31.0,2.0,1.10,10.37,10.73,1.604076e+09,
3,AE,dubai,84.20,54.0,0.0,4.70,25.26,55.30,1.604076e+09,Gulf Inn Hotel LLC
4,NG,gusau,86.85,24.0,0.0,6.67,12.16,6.66,1.604076e+09,Green Scenery Hotel
5,MV,hithadhoo,83.64,73.0,0.0,8.88,-0.60,73.08,1.604076e+09,Bloomy Inn
6,US,miramar,88.00,62.0,1.0,3.36,25.99,-80.23,1.604076e+09,Stadium Hotel
7,MX,rosamorada,85.44,48.0,1.0,1.92,22.12,-105.21,1.604076e+09,


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [38]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

locations_1 = cities_pd_clean[["Lat", "Lon"]]
humidity = cities_pd_clean["Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations_1, weights=humidity, 
                                 dissipating=True,max_intensity=100,
                                 point_radius=10)
# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Add marker layer ontop of heat map


# Display figure
